In [9]:
import itertools

In [1]:
with open('../inputs/07', 'r') as f:
    file_lines = f.readlines()
input_lines = [int(line.strip()) for line in file_lines[0].split(",")]

In [47]:
def execute_program(program, inputs):
    p = program.copy()
    i = 0
    while p[i] != 99:
        op = int(str(p[i])[-2:])
        modes = [int(char) for char in str(p[i])[:-2].zfill(3)][::-1]

        if op != 3 and op != 4:  # Compute variables
            variables = [
                p[i+1] if modes[0] else p[p[i+1]],
                p[i+2] if modes[1] else p[p[i+2]]
            ]

        if op == 1:  # Sum
            p[p[i+3]] = variables[0] + variables[1]
            i += 4
        elif op == 2:  # Multiplication
            p[p[i+3]] = variables[0] * variables[1]
            i += 4
        elif op == 3:  # Input
            p[p[i+1]] = inputs.pop(0) if len(inputs) > 0 else inputs[0]
            i += 2
        elif op == 4:  # Output
            return(p[p[i+1]])
            i += 2
        elif op == 5:
            i = variables[1] if variables[0] else i + 3
        elif op == 6:
            i = i + 3 if variables[0] else variables[1]
        elif op == 7:
            p[p[i+3]] = int(variables[0] < variables[1])
            i += 4
        elif op == 8:
            p[p[i+3]] = int(variables[0] == variables[1])
            i += 4

    return p

In [58]:
def execute_program_two(program, inputs, i=0):
    p = program.copy()
    while p[i] != 99:
        op = int(str(p[i])[-2:])
        modes = [int(char) for char in str(p[i])[:-2].zfill(3)][::-1]

        if op != 3 and op != 4:  # Compute variables
            variables = [
                p[i+1] if modes[0] else p[p[i+1]],
                p[i+2] if modes[1] else p[p[i+2]]
            ]

        if op == 1:  # Sum
            p[p[i+3]] = variables[0] + variables[1]
            i += 4
        elif op == 2:  # Multiplication
            p[p[i+3]] = variables[0] * variables[1]
            i += 4
        elif op == 3:  # Input
            p[p[i+1]] = inputs.pop(0) if len(inputs) > 0 else inputs[0]
            i += 2
        elif op == 4:  # Output
            return [i + 2, p.copy(), p[p[i+1]]]
        elif op == 5:
            i = variables[1] if variables[0] else i + 3
        elif op == 6:
            i = i + 3 if variables[0] else variables[1]
        elif op == 7:
            p[p[i+3]] = int(variables[0] < variables[1])
            i += 4
        elif op == 8:
            p[p[i+3]] = int(variables[0] == variables[1])
            i += 4

    return [-1, None, None]

In [48]:
execute_program([3,15,3,16,1002,16,10,16,1,16,15,15,4,15,99,0,0], [4,0])

4

In [49]:
def execute_programs(program, phases):
    output = 0
    for i, phase in enumerate(phases):
        output = execute_program(program, [phase, output])
        print(f"Run #{i}: {output}")
        
    return output

In [50]:
execute_programs([3,26,1001,26,-4,26,3,27,1002,27,2,27,1,27,26,27,4,27,1001,28,-1,28,1005,28,6,99,0,0,5], [9,8,7,6,5])

Run #0: 5
Run #1: 14
Run #2: 31
Run #3: 64
Run #4: 129


129

In [11]:
combinations = list(itertools.permutations([0,1,2,3,4], 5))
combinations

[(0, 1, 2, 3, 4),
 (0, 1, 2, 4, 3),
 (0, 1, 3, 2, 4),
 (0, 1, 3, 4, 2),
 (0, 1, 4, 2, 3),
 (0, 1, 4, 3, 2),
 (0, 2, 1, 3, 4),
 (0, 2, 1, 4, 3),
 (0, 2, 3, 1, 4),
 (0, 2, 3, 4, 1),
 (0, 2, 4, 1, 3),
 (0, 2, 4, 3, 1),
 (0, 3, 1, 2, 4),
 (0, 3, 1, 4, 2),
 (0, 3, 2, 1, 4),
 (0, 3, 2, 4, 1),
 (0, 3, 4, 1, 2),
 (0, 3, 4, 2, 1),
 (0, 4, 1, 2, 3),
 (0, 4, 1, 3, 2),
 (0, 4, 2, 1, 3),
 (0, 4, 2, 3, 1),
 (0, 4, 3, 1, 2),
 (0, 4, 3, 2, 1),
 (1, 0, 2, 3, 4),
 (1, 0, 2, 4, 3),
 (1, 0, 3, 2, 4),
 (1, 0, 3, 4, 2),
 (1, 0, 4, 2, 3),
 (1, 0, 4, 3, 2),
 (1, 2, 0, 3, 4),
 (1, 2, 0, 4, 3),
 (1, 2, 3, 0, 4),
 (1, 2, 3, 4, 0),
 (1, 2, 4, 0, 3),
 (1, 2, 4, 3, 0),
 (1, 3, 0, 2, 4),
 (1, 3, 0, 4, 2),
 (1, 3, 2, 0, 4),
 (1, 3, 2, 4, 0),
 (1, 3, 4, 0, 2),
 (1, 3, 4, 2, 0),
 (1, 4, 0, 2, 3),
 (1, 4, 0, 3, 2),
 (1, 4, 2, 0, 3),
 (1, 4, 2, 3, 0),
 (1, 4, 3, 0, 2),
 (1, 4, 3, 2, 0),
 (2, 0, 1, 3, 4),
 (2, 0, 1, 4, 3),
 (2, 0, 3, 1, 4),
 (2, 0, 3, 4, 1),
 (2, 0, 4, 1, 3),
 (2, 0, 4, 3, 1),
 (2, 1, 0, 3, 4),
 (2, 1, 0,

In [51]:
possibilities = {combination: execute_programs(input_lines, combination) for combination in combinations}

Run #0: 17
Run #1: 85
Run #2: 440
Run #3: 3573
Run #4: 28604
Run #0: 17
Run #1: 85
Run #2: 440
Run #3: 3540
Run #4: 28373
Run #0: 17
Run #1: 85
Run #2: 733
Run #3: 3680
Run #4: 29460
Run #0: 17
Run #1: 85
Run #2: 733
Run #3: 5884
Run #4: 29435
Run #0: 17
Run #1: 85
Run #2: 700
Run #3: 3515
Run #4: 28173
Run #0: 17
Run #1: 85
Run #2: 700
Run #3: 5653
Run #4: 28280
Run #0: 17
Run #1: 100
Run #2: 500
Run #3: 4053
Run #4: 32444
Run #0: 17
Run #1: 100
Run #2: 500
Run #3: 4020
Run #4: 32213
Run #0: 17
Run #1: 100
Run #2: 853
Run #3: 4265
Run #4: 34140
Run #0: 17
Run #1: 100
Run #2: 853
Run #3: 6844
Run #4: 34220
Run #0: 17
Run #1: 100
Run #2: 820
Run #3: 4100
Run #4: 32853
Run #0: 17
Run #1: 100
Run #2: 820
Run #3: 6613
Run #4: 33065
Run #0: 17
Run #1: 189
Run #2: 945
Run #3: 4740
Run #4: 37940
Run #0: 17
Run #1: 189
Run #2: 945
Run #3: 7580
Run #4: 37915
Run #0: 17
Run #1: 189
Run #2: 960
Run #3: 4800
Run #4: 38420
Run #0: 17
Run #1: 189
Run #2: 960
Run #3: 7700
Run #4: 38500
Run #0: 17
Run

In [17]:
max(possibilities)

38500

In [53]:
[(k,v) for k, v in possibilities.items() if v == max(possibilities.values())]

[((0, 3, 2, 4, 1), 38500)]

In [80]:
def execute_programs_two(program, phases):
    amps = [program.copy(), program.copy(), program.copy(), program.copy(), program.copy()]
    amps_i = [0,0,0,0,0]
    amps_active = list(range(0, 5))
    output = 0
    result = None
    while len(amps_active) > 0:
        for amp in amps_active:
            program = amps[amp].copy()
            i = amps_i[amp]
            phase = phases[amp]
            program_input = [phase, output] if i == 0 else [output]
            amp_i, amp_p, output = execute_program_two(program, program_input, i)
            print(output)
            if output is not None:
                result = output
            if amp_i == -1:
                amps_active.remove(amp)
            else:
                amps[amp] = amp_p.copy()
                amps_i[amp] = amp_i
            

    return result

In [81]:
a = execute_programs_two([3, 26, 1001, 26, -4, 26, 3, 27, 1002, 27, 2, 27, 1, 27, 26,
                      27, 4, 27, 1001, 28, -1, 28, 1005, 28, 6, 99, 0, 0, 5], [9, 8, 7, 6, 5])

5
14
31
64
129
263
530
1063
2128
4257
8519
17042
34087
68176
136353
272711
545426
1090855
2181712
4363425
8726855
17453714
34907431
69814864
139629729
None
None
None
None
None


In [82]:
a

139629729

In [83]:
combinations_two = list(itertools.permutations([9,8,7,6,5], 5))
combinations_two

[(9, 8, 7, 6, 5),
 (9, 8, 7, 5, 6),
 (9, 8, 6, 7, 5),
 (9, 8, 6, 5, 7),
 (9, 8, 5, 7, 6),
 (9, 8, 5, 6, 7),
 (9, 7, 8, 6, 5),
 (9, 7, 8, 5, 6),
 (9, 7, 6, 8, 5),
 (9, 7, 6, 5, 8),
 (9, 7, 5, 8, 6),
 (9, 7, 5, 6, 8),
 (9, 6, 8, 7, 5),
 (9, 6, 8, 5, 7),
 (9, 6, 7, 8, 5),
 (9, 6, 7, 5, 8),
 (9, 6, 5, 8, 7),
 (9, 6, 5, 7, 8),
 (9, 5, 8, 7, 6),
 (9, 5, 8, 6, 7),
 (9, 5, 7, 8, 6),
 (9, 5, 7, 6, 8),
 (9, 5, 6, 8, 7),
 (9, 5, 6, 7, 8),
 (8, 9, 7, 6, 5),
 (8, 9, 7, 5, 6),
 (8, 9, 6, 7, 5),
 (8, 9, 6, 5, 7),
 (8, 9, 5, 7, 6),
 (8, 9, 5, 6, 7),
 (8, 7, 9, 6, 5),
 (8, 7, 9, 5, 6),
 (8, 7, 6, 9, 5),
 (8, 7, 6, 5, 9),
 (8, 7, 5, 9, 6),
 (8, 7, 5, 6, 9),
 (8, 6, 9, 7, 5),
 (8, 6, 9, 5, 7),
 (8, 6, 7, 9, 5),
 (8, 6, 7, 5, 9),
 (8, 6, 5, 9, 7),
 (8, 6, 5, 7, 9),
 (8, 5, 9, 7, 6),
 (8, 5, 9, 6, 7),
 (8, 5, 7, 9, 6),
 (8, 5, 7, 6, 9),
 (8, 5, 6, 9, 7),
 (8, 5, 6, 7, 9),
 (7, 9, 8, 6, 5),
 (7, 9, 8, 5, 6),
 (7, 9, 6, 8, 5),
 (7, 9, 6, 5, 8),
 (7, 9, 5, 8, 6),
 (7, 9, 5, 6, 8),
 (7, 8, 9, 6, 5),
 (7, 8, 9,

In [84]:
possibilities_two = {combination: execute_programs_two(input_lines, combination) for combination in combinations_two}

1
2
4
6
7
14
15
17
34
35
37
38
39
78
156
158
316
317
319
638
1276
2552
2553
5106
5108
10216
10217
20434
20436
20437
20439
20441
40882
81764
81766
163532
327064
654128
654130
1308260
2616520
2616521
2616522
2616524
2616525
5233050
10466100
10466101
20932202
20932204
None
None
None
None
None
1
2
4
5
7
14
15
17
18
36
38
39
40
80
160
162
324
325
650
652
1304
2608
2609
2611
5222
10444
10445
20890
20891
20893
20895
20897
41794
41796
83592
167184
334368
668736
1337472
1337474
2674948
2674949
2674950
2674951
2674953
5349906
10699812
10699813
10699815
21399630
None
None
None
None
None
1
2
4
6
7
14
15
30
32
33
35
36
72
73
146
148
296
298
299
598
1196
2392
4784
4785
4787
9574
9575
9577
19154
19155
19157
19159
38318
76636
76638
153276
306552
306554
613108
1226216
2452432
2452433
2452435
2452436
2452437
4904874
9809748
19619496
19619497
19619499
None
None
None
None
None
1
2
4
5
7
14
15
30
31
33
35
36
72
144
145
147
294
296
592
593
1186
2372
4744
4746
4747
9494
9495
9497
9498
18996
18998
19000
38000

2790793
2790795
5581590
5581591
11163182
11163184
22326368
None
None
None
None
None
0
2
4
5
6
7
9
18
36
37
38
39
78
80
160
320
321
323
325
650
1300
1301
2602
5204
5206
5207
10414
10416
20832
20833
20835
41670
83340
83342
83344
166688
333376
333378
666756
1333512
1333513
1333514
1333516
2667032
2667033
5334066
5334067
10668134
21336268
21336270
None
None
None
None
None
0
2
4
5
6
7
9
18
19
38
39
40
80
160
162
324
325
327
654
656
1312
1313
2626
2628
5256
5257
10514
10516
10517
21034
21036
42072
84144
84146
84148
168296
336592
336594
673188
1346376
1346377
1346378
1346380
1346381
2692762
5385524
5385525
10771050
10771052
21542104
None
None
None
None
None
0
2
3
4
6
7
9
10
20
40
41
42
84
86
172
344
345
690
692
694
1388
1389
1391
2782
5564
5565
11130
11131
22262
22264
22266
44532
44534
44536
89072
178144
356288
712576
1425152
1425154
1425155
1425156
1425157
2850314
2850316
5700632
5700633
5700635
11401270
22802540
None
None
None
None
None
0
2
3
5
6
7
9
10
20
40
41
42
84
168
170
340
341
682
68

112614
225228
225230
450460
900920
1801840
1801841
1801843
1801844
1801845
3603690
3603691
7207382
14414764
14414766
28829532
None
None
None
None
None
2
4
5
6
12
14
28
29
58
59
60
120
240
242
243
244
246
492
494
988
989
1978
1980
3960
7920
15840
15842
15843
31686
31687
63374
126748
126750
126752
126754
253508
253510
507020
1014040
2028080
2028081
2028083
2028084
4056168
4056169
4056170
8112340
8112342
16224684
32449368
None
None
None
None
None
2
4
5
10
11
13
26
27
28
56
57
114
228
229
231
232
234
468
936
938
939
1878
1880
3760
7520
15040
15042
15043
15044
30088
60176
120352
120354
120356
120358
240716
240718
481436
962872
1925744
1925745
1925747
1925748
1925749
3851498
3851499
7702998
7703000
15406000
30812000
None
None
None
None
None
2
3
4
8
10
12
13
26
27
54
55
110
112
113
226
227
454
456
912
914
915
917
1834
3668
7336
14672
14673
29346
29347
29349
58698
58700
58702
58704
117408
234816
469632
939264
1878528
1878530
1878531
1878532
3757064
3757065
3757067
3757068
3757070
7514140
15028

1723
3446
6892
6894
6895
13790
13791
27582
55164
55166
110332
110334
110336
110338
220676
441352
882704
1765408
1765410
1765411
1765412
1765413
3530826
7061652
7061654
7061655
14123310
28246620
None
None
None
None
None
1
2
4
6
8
9
18
19
21
42
84
86
87
88
176
352
354
708
709
711
713
1426
2852
2853
5706
5707
11414
11415
22830
22832
22834
22836
22838
45676
91352
182704
365408
730816
1461632
1461634
1461635
2923270
2923271
2923272
2923274
2923276
5846552
11693104
11693105
23386210
None
None
None
None
None
1
2
4
6
8
9
18
19
38
40
80
82
83
166
167
334
336
672
674
675
677
1354
2708
5416
5417
5418
10836
10837
10839
21678
21680
21682
21684
43368
86736
173472
346944
693888
693890
1387780
1387781
2775562
2775563
2775565
2775566
2775568
5551136
11102272
22204544
22204545
None
None
None
None
None
1
2
4
8
10
11
22
24
25
50
100
102
103
104
208
416
418
419
838
840
842
1684
1685
3370
6740
6741
13482
26964
26965
26967
26969
26971
53942
53944
107888
215776
431552
863104
1726208
1726210
1726211
3452422
34

In [85]:
[(k,v) for k, v in possibilities_two.items() if v == max(possibilities_two.values())]

[((7, 5, 9, 6, 8), 33660560)]